**Importing Libraries**

In [261]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml.html as lh
import requests

**Scraping Wikipage**

In [262]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

**Transforming data from table into DataFrame**

In [266]:
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [267]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data = t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [268]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [269]:
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


**Modifying column order**

In [270]:
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]

In [271]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


**Renaming columns**

In [272]:
df.rename(index = str, columns = {"Postcode" :"PostalCode", "Neighbourhood\n" : "Neighbourhood"}, inplace = True)

In [273]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


**Removing Not Assigned**

In [274]:
df.drop(df.loc[df['Borough']=="Not assigned"].index, inplace=True)

In [275]:
df.reset_index(drop=True, inplace=True)

In [276]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


**Removing \n in Neighbourhood**

In [277]:
i = 0
temp_list = []
for index, row in df.iterrows():
    temp_list.append(df['Neighbourhood'][i].replace('\n','',True))
    i+=1  

In [278]:
len(temp_list)

211

In [279]:
df['Neighbourhood'] = temp_list

In [280]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**Grouping by Neighbourhood**

In [281]:
df1 = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [282]:
df1.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [283]:
df1.shape

(103, 3)